In [ ]:
# ==== CONFIG ====
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# ==== CONFIG ====

CSV_PATH = '/content/drive/MyDrive/thesis/combined_data_Auto_pilot.csv'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
EPOCHS = 20
LATENT_DIM = 10  # Noise input dimension for GAN

# ==== Dataset ====
class GPSSpoofDataset(Dataset):
    def __init__(self, csv_path):
        data = pd.read_csv(csv_path)

        # Select only required columns
        feature_columns = [
            'accel_x', 'accel_y', 'accel_z',
            'gyro_x', 'gyro_y', 'gyro_z',
            'speed', 'steering_angle', 'throttle', 'brake'
        ]
        self.X = data[feature_columns].astype(np.float32).values

        # Convert label to 0 (normal) or 1 (spoofed)
        self.y = (data['label'] == 'spoofed').astype(int).values

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

dataset = GPSSpoofDataset(CSV_PATH)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# ==== Classifier ====
class SensorClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.net(x)

# ==== GAN ====
class Generator(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 10)  # Output shape matches sensor features
        )

    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(10, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# ==== Initialize Models ====
classifier = SensorClassifier().to(DEVICE)
generator = Generator().to(DEVICE)
discriminator = Discriminator().to(DEVICE)

opt_C = optim.Adam(classifier.parameters(), lr=1e-4)
opt_G = optim.Adam(generator.parameters(), lr=1e-4)
opt_D = optim.Adam(discriminator.parameters(), lr=1e-4)

loss_cls = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 1.5]).to(DEVICE))
loss_adv = nn.BCELoss()

# ==== Training Loop ====
print("🛡️ Training classifier with adversarial sensor samples...")

for epoch in range(EPOCHS):
    classifier.train()
    generator.train()
    discriminator.train()

    total_loss, correct, total = 0, 0, 0

    for real_data, labels in dataloader:
        real_data = real_data.to(DEVICE)
        labels = labels.to(DEVICE)

        # === 1. Train Discriminator ===
        z = torch.randn(real_data.size(0), LATENT_DIM).to(DEVICE)
        fake_data = generator(z)

        d_real = discriminator(real_data)
        d_fake = discriminator(fake_data.detach())
        loss_d = loss_adv(d_real, torch.ones_like(d_real)) + \
                 loss_adv(d_fake, torch.zeros_like(d_fake))

        opt_D.zero_grad()
        loss_d.backward()
        opt_D.step()

        # === 2. Train Generator ===
        d_fake = discriminator(fake_data)
        loss_g = loss_adv(d_fake, torch.ones_like(d_fake))  # fool the discriminator

        opt_G.zero_grad()
        loss_g.backward()
        opt_G.step()

        # === 3. Train Classifier (on real + fake samples) ===
        spoof_labels = torch.ones(real_data.size(0), dtype=torch.long).to(DEVICE)
        fake_data_for_cls = generator(torch.randn(real_data.size(0), LATENT_DIM).to(DEVICE))

        # Mix real and adversarial spoofed data
        data_cls = torch.cat([real_data, fake_data_for_cls], dim=0)
        label_cls = torch.cat([labels, spoof_labels], dim=0)

        pred = classifier(data_cls)
        loss_c = loss_cls(pred, label_cls)

        opt_C.zero_grad()
        loss_c.backward()
        opt_C.step()

        total_loss += loss_c.item()
        pred_cls = pred.argmax(dim=1)
        correct += (pred_cls == label_cls).sum().item()
        total += label_cls.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss:.4f} - Accuracy: {acc:.2f}%")

# ==== Save the trained classifier ====
torch.save(classifier.state_dict(), 'aml_gps_classifier.pth')
print("✅ Classifier saved as aml_gps_classifier.pth")




=== Spoofing Control ===
Type 's' + ENTER → Activate spoofing
Type 'n' + ENTER → Deactivate spoofing
Type 'q' + ENTER → Quit simulation

[Frame 0] Prediction: normal (0.55)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


[Frame 0] Prediction: normal (0.56)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


[Frame 0] Prediction: normal (0.56)


...

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


[Frame 0] Prediction: normal (0.51)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


[Frame 0] Prediction: normal (0.52)
✅ Spoofing Deactivated.


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


In [ ]:

# === SPOOFING CONTROL ===
print("\n=== Spoofing Control ===")
print("Type 's' + ENTER → Activate spoofing")
print("Type 'n' + ENTER → Deactivate spoofing")
print("Type 'q' + ENTER → Quit simulation\n")

def spoofing_input_loop():
    global spoofing_active, running
    while running:
        command = input("Command (s=spoof, n=normal, q=quit): ").strip().lower()
        if command == 's':
            spoofing_active = True
            print("🚨 Spoofing Activated.")
        elif command == 'n':
            spoofing_active = False
            print("✅ Spoofing Deactivated.")
        elif command == 'q':
            print("🛑 Quitting simulation.")
            running = False
            break

running = True
input_thread = threading.Thread(target=spoofing_input_loop)
input_thread.start()

try:
    while running and vehicle.is_alive:
        world.tick()
        time.sleep(0.05)
except KeyboardInterrupt:
    print("🔁 Simulation interrupted by user.")
    running = False

input_thread.join()



[Frame 0] Prediction: normal (0.57)
[Frame 0] Prediction: normal (0.57)

=== Spoofing Control ===
Type 's' + ENTER → Activate spoofing
Type 'n' + ENTER → Deactivate spoofing
Type 'q' + ENTER → Quit simulation

[Frame 0] Prediction: normal (0.59)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


[Frame 0] Prediction: normal (0.52)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


[Frame 0] Prediction: normal (0.53)


...

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


[Frame 0] Prediction: normal (0.58)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


🚨 Spoofing Activated.
[Frame 0] Prediction: normal (0.74)


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - img marked as output argument, but provided NumPy array marked as readonly
>  - Expected Ptr<cv::UMat> for argument 'img'


In [ ]:

# === CLEANUP & SAVE ===
try:
    pd.DataFrame(data_records).to_csv('combined_data_Aml.csv', index=False)
    print("💾 combined_data_Aml.csv saved.")
except Exception as e:
    print(f"Error saving CSV: {e}")

for s in sensor_list:
    try:
        s.stop()
        s.destroy()
    except:
        pass

try:
    vehicle.destroy()
except:
    pass

for writer in video_writers.values():
    try:
        writer.release()
    except:
        pass

print("✅ Data and videos saved. All actors cleaned up.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, classification_report
from matplotlib.backends.backend_pdf import PdfPages
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import threading

# === PARAMETERS ===
THRESHOLD = 0.6  # Adjustable spoofing threshold
CSV_FILE = 'combined_data_Aml.csv'

# === LOAD CSV ===
data = pd.read_csv(CSV_FILE)
data['label_bin'] = data['label'].map({'normal': 0, 'spoofed': 1})
data['predicted_bin'] = (data['spoof_probability'] > THRESHOLD).astype(int)

# === EVALUATION METRICS ===
true_labels = data['label_bin'].values
predicted_labels = data['predicted_bin'].values
spoof_probs = data['spoof_probability'].values

cm = confusion_matrix(true_labels, predicted_labels)
roc_fpr, roc_tpr, _ = roc_curve(true_labels, spoof_probs)
roc_auc = auc(roc_fpr, roc_tpr)
report = classification_report(true_labels, predicted_labels, target_names=['Normal', 'Spoofed'], output_dict=True)
report_text = classification_report(true_labels, predicted_labels, target_names=['Normal', 'Spoofed'])

# === PLOT TO PDF ===
with PdfPages("spoofing_summary_report.pdf") as pdf:
    # Confusion Matrix
    fig_cm, ax_cm = plt.subplots(figsize=(6, 5))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal', 'Spoofed'])
    disp.plot(ax=ax_cm, cmap='Blues')
    plt.title("Confusion Matrix")
    pdf.savefig(fig_cm)
    plt.close()

    # ROC Curve
    plt.figure(figsize=(6, 5))
    plt.plot(roc_fpr, roc_tpr, label=f"ROC AUC = {roc_auc:.2f}", color='darkorange')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    pdf.savefig()
    plt.close()

    # Classification Report
    fig_report, ax = plt.subplots(figsize=(6, 3))
    ax.axis('off')
    ax.text(0.01, 0.99, report_text, fontsize=10, va='top', family='monospace')
    plt.title("Classification Report")
    pdf.savefig(fig_report)
    plt.close()

print("\n✅ PDF summary report saved as 'spoofing_summary_report.pdf'")

# === DASH LIVE DASHBOARD ===
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H2("Real-Time Spoofing Probability Viewer"),
    dcc.Slider(
        id='frame-slider',
        min=0,
        max=len(data)-1,
        step=1,
        value=0,
        marks={0: 'Start', len(data)-1: 'End'}
    ),
    dcc.Graph(id='spoof-prob-graph'),
    html.Div(id='frame-info', style={'marginTop': '20px', 'fontFamily': 'monospace'})
])

@app.callback(
    [Output('spoof-prob-graph', 'figure'),
     Output('frame-info', 'children')],
    [Input('frame-slider', 'value')]
)
def update_frame(frame_id):
    window = data.iloc[max(0, frame_id-50):frame_id+1]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=window['frame_id'], y=window['spoof_probability'], mode='lines+markers',
                             name='Spoof Probability', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=window['frame_id'], y=[THRESHOLD]*len(window), mode='lines',
                             name='Threshold', line=dict(color='gray', dash='dash')))
    fig.update_layout(title="Spoof Probability over Time", xaxis_title="Frame ID", yaxis_title="Probability",
                      yaxis=dict(range=[0, 1]), height=400)

    row = data.iloc[frame_id]
    info = f"Frame ID: {row['frame_id']}\nLabel: {row['label']} | Predicted: {row['predicted_label']}\nSpoof Probability: {row['spoof_probability']}"
    return fig, info

# Run the dashboard in a separate thread
threading.Thread(target=app.run_server, kwargs={'debug': False, 'use_reloader': False}).start()